In [1]:
import pandas as pd

In [2]:
AL_IMAGE = 'https://s3.amazonaws.com/openneuro.org/ds004515/participants.tsv?versionId=OX_rM1XL_frHnMPy12BwJttx3nN3k8kn'
AL_IMAGE = pd.read_csv(AL_IMAGE, sep='\t')
AL_IMAGE.to_csv('AL_IMAGEIC_dataset_ds004515.csv', sep=',', index=False)

In [4]:
suda = pd.read_csv('data_ds004515/SUDA_ds004515.csv', sep=',')
pif = pd.read_csv('data_ds004515/PIF_ds004515.csv', sep=',',index_col='Unnamed: 0').sort_index()
#suda_field_score =  pd.read_csv('data_ds004515/att_suda.csv', sep=',').drop(columns=['Unnamed: 0'], errors='ignore')
#k_combined_field =  pd.read_csv('data_ds004515/normalized_difference.csv', sep=',').drop(columns=['Difference in Unique Rows','Unique Rows After Removal']).rename(columns={'Column': 'variable'})
#k_combined_all = pd.read_csv('data_ds004515/k-combined-all-2-11.csv', sep=',')
